In [1]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import (StandardScaler, OrdinalEncoder,LabelEncoder, MinMaxScaler, OneHotEncoder)

from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model

import tensorflow as tf
from tensorflow  import keras

from keras.layers import Bidirectional
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM
from sklearn.metrics import mean_squared_error

In [17]:
# folder_path = r"C:\Users\Mohamed\OneDrive\Desktop\Solar datasets"
df = pd.read_csv(r"C:\Users\Mohamed\OneDrive\Desktop\Solar datasets\Plant_2_Generation_Data.csv")
df

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15 00:00:00,4136001,4UPUqMRk7TRMgml,0.0,0.0,9425.000000,2.429011e+06
1,2020-05-15 00:00:00,4136001,81aHJ1q11NBPMrL,0.0,0.0,0.000000,1.215279e+09
2,2020-05-15 00:00:00,4136001,9kRcWv60rDACzjR,0.0,0.0,3075.333333,2.247720e+09
3,2020-05-15 00:00:00,4136001,Et9kgGMDl729KT4,0.0,0.0,269.933333,1.704250e+06
4,2020-05-15 00:00:00,4136001,IQ2d7wF4YD8zU1Q,0.0,0.0,3177.000000,1.994153e+07
...,...,...,...,...,...,...,...
67693,2020-06-17 23:45:00,4136001,q49J1IKaHRwDQnt,0.0,0.0,4157.000000,5.207580e+05
67694,2020-06-17 23:45:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,3931.000000,1.211314e+08
67695,2020-06-17 23:45:00,4136001,vOuJvMaM2sgwLmb,0.0,0.0,4322.000000,2.427691e+06
67696,2020-06-17 23:45:00,4136001,xMbIugepa2P7lBB,0.0,0.0,4218.000000,1.068964e+08


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67698 entries, 0 to 67697
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATE_TIME    67698 non-null  object 
 1   PLANT_ID     67698 non-null  int64  
 2   SOURCE_KEY   67698 non-null  object 
 3   DC_POWER     67698 non-null  float64
 4   AC_POWER     67698 non-null  float64
 5   DAILY_YIELD  67698 non-null  float64
 6   TOTAL_YIELD  67698 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 3.6+ MB


In [3]:
# Setting the DATE_TIME column as an df index after convirting its data type from Object to datetime64

df["DATE_TIME"] = df["DATE_TIME"].astype(np.datetime64)
df.set_index("DATE_TIME",inplace=True)

In [4]:
df

,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
DATE_TIME,,,,,,
2020-05-15 00:00:00,4136001,4UPUqMRk7TRMgml,0.0,0.0,9425.000000,2.429011e+06
2020-05-15 00:00:00,4136001,81aHJ1q11NBPMrL,0.0,0.0,0.000000,1.215279e+09
2020-05-15 00:00:00,4136001,9kRcWv60rDACzjR,0.0,0.0,3075.333333,2.247720e+09
2020-05-15 00:00:00,4136001,Et9kgGMDl729KT4,0.0,0.0,269.933333,1.704250e+06
2020-05-15 00:00:00,4136001,IQ2d7wF4YD8zU1Q,0.0,0.0,3177.000000,1.994153e+07
...,...,...,...,...,...,...
2020-06-17 23:45:00,4136001,q49J1IKaHRwDQnt,0.0,0.0,4157.000000,5.207580e+05
2020-06-17 23:45:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,3931.000000,1.211314e+08
2020-06-17 23:45:00,4136001,vOuJvMaM2sgwLmb,0.0,0.0,4322.000000,2.427691e+06


In [5]:
# Dropping "PLANT_ID" , "SOURCE_KEY" object columns 

df.drop(["PLANT_ID" , "SOURCE_KEY"],axis=1,inplace=True)

In [6]:
# resampling the rows by day instead of taking a sample each 15 min

df = df.resample('D').sum()

df

,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
DATE_TIME,,,,
2020-05-15,666607.630952,651437.736667,7.997412e+06,1.347430e+12
2020-05-16,566839.840000,554253.460476,7.234658e+06,1.346965e+12
2020-05-17,556236.918205,544393.020916,6.712265e+06,1.357864e+12
2020-05-18,530773.677143,519012.565238,6.259087e+06,1.361871e+12
2020-05-19,443495.847015,433554.722647,5.177354e+06,1.314653e+12
2020-05-20,352374.378810,344661.406190,5.764847e+06,1.146138e+12
2020-05-21,494112.260000,483021.364286,5.674293e+06,1.258768e+12
2020-05-22,498711.327619,487411.941136,5.897264e+06,1.274881e+12
2020-05-23,501823.294762,490450.510476,6.287109e+06,1.258321e+12


In [7]:
# Data rang is huge , so we're gonna scale it

def scale_data(data):
    scaler = MinMaxScaler().fit(data)
    return scaler.transform(data) ,scaler

data, scaler= scale_data(df)
data = pd.DataFrame(data)

In [8]:
# splitting a given series x , n using a given number of steps (How many value at each input)

def univariate_splitter(series,steps):
    X = []
    y = []

    len_series = len(series)

    for i in range(len_series):

        end_idx = i + steps

        if end_idx > len_series - 1:
            break

        input_x, output_y = series[i:end_idx], series[end_idx: end_idx + 1]

        X.append(input_x)
        y.append(output_y)

    return np.array(X), np.mean(np.array(y), axis=1)

In [9]:
# we're spliiting the data into x,y using the 3rd column "DAILY_YIELD" and 6 steps (6 days per each sample)

X,y = univariate_splitter(data.iloc[:,2],6)

In [10]:
# Reshaping the array from 2 dim array to 3 dim array to be ready to passed into the model

X = X.reshape(X.shape[0],X.shape[1],1)
X.shape

(28, 6, 1)

In [11]:
blstm_model = Sequential()
blstm_model.add(Bidirectional(LSTM(128, activation='relu', input_shape=(X.shape[1], X.shape[2]), return_sequences=True)))
blstm_model.add(Bidirectional(LSTM(64, activation='relu')))
blstm_model.add(Dense(32, activation='relu'))
blstm_model.add(Dropout(0.2))
blstm_model.add(Dense(1))

In [12]:
# Compiling the model using mse loss function

blstm_model.compile(loss='mean_squared_error', optimizer='adam')

In [13]:
# Setting the EarlyStopping function and fitting the data to the model

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

history = blstm_model.fit(X, y, epochs= 100, verbose= 1, callbacks= callback)

Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 0.3003
Epoch 2/100
1/1 [==============================] - 0s 13ms/step - loss: 0.2804
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 0.2642
Epoch 4/100
1/1 [==============================] - 0s 12ms/step - loss: 0.2526
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 0.2335
Epoch 6/100
1/1 [==============================] - 0s 15ms/step - loss: 0.2063
Epoch 7/100
1/1 [==============================] - 0s 14ms/step - loss: 0.1859
Epoch 8/100
1/1 [==============================] - 0s 14ms/step - loss: 0.1659
Epoch 9/100
1/1 [==============================] - 0s 13ms/step - loss: 0.1510
Epoch 10/100
1/1 [==============================] - 0s 13ms/step - loss: 0.1346
Epoch 11/100
1/1 [==============================] - 0s 14ms/step - loss: 0.1045
Epoch 12/100
1/1 [==============================] - 0s 13ms/step - loss: 0.0909
Epoch 13/100
1/1 [==============================] -

In [14]:
blstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 6, 256)           133120    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 301,633
Trainable params: 301,633
Non-trai

In [15]:
def root_mean_squared_error(y_true, y_pred):    
    return np.sqrt(mean_squared_error(y_true, y_pred))

print('Train RMSE   : %.3f ' % root_mean_squared_error(y, blstm_model.predict(X)))
print('Train MSE    : %.3f ' % mean_squared_error(y, blstm_model.predict(X)))
print('---------------------------------------------')

1/1 [==============================] - 1s 640ms/step
Train RMSE   : 0.236 
1/1 [==============================] - 0s 27ms/step
Train MSE    : 0.056 
---------------------------------------------
